# GRS

In [3]:
from collections import OrderedDict

import jinja2
import os
import pandas as pd
#from scipy.optimize import minimize
import statsmodels.api as sm

#%matplotlib inline

C:\Users\samth\Miniconda3\envs\econ36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
factors = pd.read_csv("C:/Data/Thesis/_Factors.csv")

myfactors = pd.read_csv("C:/Data/Thesis/_MyFactors.csv")
myfactors.iloc[:,1:] = myfactors.iloc[:,1:]*100

f = pd.concat([factors, myfactors[["HML", "HMLm", "PMU06", "PMU16", "PMU", "CMA", "WML"]]], axis=1)
f.date = pd.to_datetime(f.date.astype("str"), format="%Y%m")
f = f.set_index("date")

f.head()

,rf,rm,smb,hml,rmw,cma,wml,str,hmlm,bab,qmj,HML,HMLm,PMU06,PMU16,PMU,CMA,WML
date,,,,,,,,,,,,,,,,,,
1963-07-01,0.27,-0.39,-0.46,-0.81,0.72,-1.16,0.99,-0.04,-0.59,0.81,0.47,-0.721155,-0.864465,0.888426,0.739828,0.355650,-1.132040,0.550726
1963-08-01,0.25,5.07,-0.81,1.65,0.42,-0.40,1.08,1.36,0.28,-1.22,-0.63,1.000628,-0.060107,0.212301,0.470122,0.456718,-0.070141,1.443902
1963-09-01,0.27,-1.57,-0.48,0.19,-0.80,0.23,0.13,1.20,0.27,1.07,0.61,1.045719,1.463823,-1.169019,-1.130309,-0.665664,-0.361107,0.267052
1963-10-01,0.29,2.53,-1.29,-0.09,2.75,-2.26,3.14,-0.66,-2.11,-1.88,2.08,-0.549914,-1.635921,2.612164,3.006009,3.007174,-1.886318,3.648531
1963-11-01,0.27,-0.85,-0.85,1.71,-0.34,2.22,-0.75,1.15,0.91,-0.52,-0.88,1.288533,1.400487,-0.099125,-0.481188,-0.625498,0.314912,-1.489579


In [5]:
f93 = ["rm", "smb", "hml"]
c97  = ["rm", "smb", "hml", "wml"]
c97b = ["rm", "smb", "hmlm", "wml"]
f15 = ["rm", "smb", "hml", "rmw", "cma"]
f16 = ["rm", "smb", "hml", "PMU", "cma"]
f17 = ["rm", "smb", "hml", "PMU", "cma", "wml"]
b16  = ["rm", "smb", "hmlm", "wml", "PMU"]
b16b = ["rm", "smb", "hmlm", "wml", "PMU", "cma"]

In [6]:
def Sh2(f):
    Ef = pd.np.mean(f, axis=0)
    CoVf = pd.np.cov(f, rowvar=False)
    return Ef.transpose() @ pd.np.linalg.pinv(CoVf) @ Ef

In [7]:
def GRS(R, F, factors, start="1963-07", end="2017-12"):
    """
    Returns the GRS statistic of Gibbons, Ross and Shanken (1989)
    """

    alphas = []
    errors = []

    for p in R.columns:
        model = sm.OLS(R[start:end][p] - F[start:end]["rf"],
                       sm.add_constant(F[start:end][factors]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        errors.append(fit.resid)

    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()

    K = F[factors].shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)

    r = F[factors].mean().as_matrix()

    f = F[factors].as_matrix()

    Vf = (f-r).transpose().dot(f-r) / (T - 1)

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f

    return (T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))

In [8]:
path = "C:/Users/samth/Dropbox/Thesis/Tex/Tables/"

env = jinja2.Environment(
    block_start_string='-%', block_end_string='%-',
    variable_start_string='=%', variable_end_string='%=',
    loader=jinja2.FileSystemLoader(path)
)

In [9]:
n_months, start, end = 654, "1963-07", "2017-12"

## Anomalies

We need to combine some anomalies.

* $BM$ constrained
* Value constrained
* Others
* All

In [52]:
BM_keys    = [r"Size-BM",
              r"Size-BM-Inv",
              r"Size-BM-$\text{OP}^{06}$"]
Value_keys = BM_keys + [r"Size-$\text{BM}^m$-Prior",]
Other_keys = [r"Size-$\text{OP}^{06}$",
              r"Size-Inv",
              r"Size-$\text{OP}^{06}$-Inv",
              r"Size-Acc",
              r"Size-$\beta$",
              r"Size-NI",
              r"Size-Prior",
              r"Size-RVar",
              r"Size-Var"]
All_keys   = Value_keys + Other_keys

anomalies = OrderedDict([
    (r"Size-BM", {"path": "C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"}),
    (r"Size-BM-Inv", {"path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV"}),
    (r"Size-BM-$\text{OP}^{06}$", {"path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV"}),
])

pd.concat(
    [pd.read_csv(anomalies[a]["path"]).iloc[:, 1:] for a in BM_keys],
    axis=1
).to_csv("C:/Data/Thesis/89_BM_Portfolios.csv")
anomalies[r"BM constrained"] = {"path": "C:/Data/Thesis/89_BM_Portfolios.csv"}

anomalies[r"Size-$\text{BM}^m$-Prior"] = {"path": "C:/Data/Thesis/32_Portfolios_BMm_Prior.csv"}

pd.concat(
    [pd.read_csv(anomalies[a]["path"]).iloc[:, 1:] for a in Value_keys],
    axis=1
).to_csv("C:/Data/Thesis/121_BM_Portfolios.csv")
anomalies[r"Value constrained"] = {"path": "C:/Data/Thesis/121_BM_Portfolios.csv"}

anomalies[r"Size-$\text{OP}^{06}$"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_OP_5x5.CSV"}
anomalies[r"Size-Inv"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV"}
anomalies[r"Size-$\text{OP}^{06}$-Inv"] = {"path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV"}
anomalies[r"Size-Acc"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_AC_5x5.CSV"}
anomalies[r"Size-$\beta$"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.CSV"}
anomalies[r"Size-NI"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_NI_5x5.CSV"}
anomalies[r"Size-Prior"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV"}
anomalies[r"Size-RVar"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_RESVAR_5x5.CSV"}
anomalies[r"Size-Var"] = {"path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.CSV"}

pd.concat(
    [pd.read_csv(anomalies[a]["path"]).iloc[:, 1:] for a in Other_keys],
    axis=1
).to_csv("C:/Data/Thesis/242_Other_Portfolios.csv")
anomalies[r"Selected anomalies"] = {"path": "C:/Data/Thesis/242_Other_Portfolios.csv"}

pd.concat(
    [pd.read_csv(anomalies[a]["path"]).iloc[:, 1:] for a in All_keys],
    axis=1
).to_csv("C:/Data/Thesis/363_Portfolios.csv")
anomalies[r"All"] = {"path": "C:/Data/Thesis/363_Portfolios.csv"}

In [53]:
%%time

for a in anomalies:
    df = pd.read_csv(anomalies[a]["path"])
    df.index = f.index

    # remove date column from anomalies and the index column from combinations of anomalies
    df = df.iloc[:, 1:]
    df = df.fillna(-99.99)

    print("{:.<35s}{:4d}, {:4d}".format(a, df.shape[0], df.shape[1]))

    anomalies[a]["GRS"] = OrderedDict([
        ("C97",  "{:.2f}".format(GRS(df, f, c97))),
        ("C97b", "{:.2f}".format(GRS(df, f, c97b))),
        ("F15",  "{:.2f}".format(GRS(df, f, f15))),
        ("F16",  "{:.2f}".format(GRS(df, f, f16))),
        ("F17",  "{:.2f}".format(GRS(df, f, f17))),
        ("B16",  "{:.2f}".format(GRS(df, f, b16))),
        ("B16b", "{:.2f}".format(GRS(df, f, b16b))),
    ])

Size-BM............................ 654,   25
Size-BM-Inv........................ 654,   32
Size-BM-$\text{OP}^{06}$........... 654,   32
BM constrained..................... 654,   89
Size-$\text{BM}^m$-Prior........... 654,   32
Value constrained.................. 654,  121
Size-$\text{OP}^{06}$.............. 654,   25
Size-Inv........................... 654,   25
Size-$\text{OP}^{06}$-Inv.......... 654,   32
Size-Acc........................... 654,   25
Size-$\beta$....................... 654,   25
Size-NI............................ 654,   35
Size-Prior......................... 654,   25
Size-RVar.......................... 654,   25
Size-Var........................... 654,   25
Selected anomalies................. 654,  242
All................................ 654,  363
Wall time: 54.2 s


In [84]:
template = env.get_template("_GRS_template.txt")

In [85]:
context = {
    "caption": r"GRS Statistic and $Sh^2$ -- 1963-07 through 2017-12",
    "label": "tbl:GRS",
    "model_names": ["C97", "C97b", "F15", "F16", "F17", "B16", "B16b"],
    "anomalies": anomalies,
}

In [86]:
with open(os.path.join(path, "GRS_196307_201712.tex"), "w") as table:
    table.write(template.render(context))

# Annual value with monthly value

In [8]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV")
a.date = pd.to_datetime(a.date.astype(str), format="%Y%m")
a = a.set_index("date")
GRS(a, f, ["rm", "smb", "hmlm", "wml", "PMU"]), GRS(a, f, ["rm", "smb", "hmlm", "hml", "wml", "PMU"])

(2.5754522402947893, 2.4333800340076737)

In [9]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV")
a.date = pd.to_datetime(a.date.astype(str), format="%Y%m")
a = a.set_index("date")
GRS(a, f, ["rm", "smb", "hmlm", "wml", "PMU"]), GRS(a, f, ["rm", "smb", "hmlm", "hml", "wml", "PMU"])

(2.1303728973157794, 1.9738204529827585)

In [10]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV")
a.date = pd.to_datetime(a.date.astype(str), format="%Y%m")
a = a.set_index("date")
GRS(a, f, ["rm", "smb", "hmlm", "wml", "PMU"]), GRS(a, f, ["rm", "smb", "hmlm", "hml", "wml", "PMU"])

(1.5669420938109266, 1.4020879587999056)

# Liquidity-type stuff

In [13]:
a = pd.read_csv("C:/Data/Thesis/32_Portfolios_BMm_Prior.csv").iloc[:,1:]
a = a.fillna(-99.99)
a.index = f.index
[round(grs, 2) for grs in
     [GRS(a, f, b16),
      GRS(a, f, ["rm", "hmlm", "wml", "PMU"]),
      GRS(a, f, b16+["hml",]),
      GRS(a, f, b16+["str",]),
      GRS(a, f, b16+["bab",]),
      GRS(a, f, b16+["qmj",])]]

[4.85, 5.31, 4.81, 4.69, 5.02, 5.4]

In [12]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV").iloc[:,1:]
a = a.fillna(-99.99)
a.index = f.index
[round(grs, 2) for grs in
     [GRS(a, f, b16),
      GRS(a, f, ["rm", "hmlm", "wml", "PMU"]),
      GRS(a, f, b16+["hml",]),
      GRS(a, f, b16+["str",]),
      GRS(a, f, b16+["bab",]),
      GRS(a, f, b16+["qmj",]),
      GRS(a, f, b16+["str", "bab"]),]]

[3.62, 4.04, 4.0, 3.36, 3.45, 3.94, 3.19]

In [11]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv").iloc[:,1:]
a = a.fillna(-99.99)
a.index = f.index
[round(grs, 2) for grs in
     [GRS(a, f, b16),
      GRS(a, f, ["rm", "hmlm", "wml", "PMU"]),
      GRS(a, f, b16+["hml",]),
      GRS(a, f, b16+["str",]),
      GRS(a, f, b16+["bab",]),
      GRS(a, f, b16+["qmj",]),
      GRS(a, f, b16+["str", "bab"]),]]

[1.15, 1.65, 1.3, 1.07, 1.2, 1.57, 1.12]

In [10]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv").iloc[:,1:]
a = a.fillna(-99.99)
a.index = f.index
[round(grs, 2) for grs in
     [GRS(a, f, b16),
      GRS(a, f, ["rm", "hmlm", "wml", "PMU"]),
      GRS(a, f, b16+["hml",]),
      GRS(a, f, b16+["str",]),
      GRS(a, f, b16+["bab",]),
      GRS(a, f, b16+["qmj",]),
      GRS(a, f, b16+["str", "bab"]),]]

[4.13, 4.7, 4.13, 4.01, 3.8, 4.08, 3.67]